In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.ByteString                  as B
import qualified Data.ByteString.Char8            as B (lines)
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

input <- B.lines <$> B.readFile "../input/day10.txt"

In [2]:
data Bot         = Bot Int                              deriving (Eq, Show, Ord)
data Sink        = OtherBot  Bot | Bucket Int           deriving (Eq, Show)
data Instruction = Input Int Bot | Output Bot Sink Sink deriving (Eq, Show)

data Storage = Holding (Maybe Int) (Maybe Int) deriving (Eq, Show)

parseInput = do
    value <- "value " *> AC.decimal <* " goes to bot "
    botN  <- AC.decimal
    return $ Input value (Bot botN)

parseSink = A.choice
    [ OtherBot . Bot <$> ("bot "    *> AC.decimal)
    , Bucket         <$> ("output " *> AC.decimal)
    ]

parseOutput = Output
    <$> (Bot <$> ("bot " *> AC.decimal))
    <*> (" gives low to " *> parseSink)
    <*> (" and high to "  *> parseSink)

parseInstruction = A.choice [parseInput, parseOutput]

parsed = map (either error id . A.parseOnly parseInstruction) input

In [3]:
import qualified Data.Map.Strict as M

type Stored  = M.Map Bot Storage
type Outputs = M.Map Int [Int]
data State   = State Stored Outputs [Instruction]

full :: Bot -> Stored -> Bool
full bot stored = case M.lookup bot stored of
    Just (Holding (Just _) (Just _)) -> True
    _                                -> False

storeInput :: Bot -> Stored -> Int -> Stored
storeInput bot stored int = case M.lookup bot stored of
    Nothing                         -> M.insert bot (Holding (Just         int) Nothing)            stored
    Just (Holding (Just x) Nothing) -> M.insert bot (Holding (Just $ min x int) (Just $ max x int)) stored

putBucket :: Int -> Int -> Outputs -> Outputs
putBucket bucketNo chip outputs = case M.lookup bucketNo outputs of
    Nothing    -> M.insert bucketNo [chip]       outputs
    Just chips -> M.insert bucketNo (chip:chips) outputs

output :: Storage -> Sink -> Sink -> Stored -> Outputs -> (Stored, Outputs)
output (Holding (Just mn) (Just mx)) sink1 sink2 stored outputs = let
    (stored',outputs') = case sink1 of
        OtherBot bot -> (storeInput bot stored mn, outputs)
        Bucket b     -> (stored, putBucket b mn outputs)
    (stored'',outputs'') = case sink2 of
        OtherBot bot' -> (storeInput bot' stored' mx, outputs')
        Bucket b'     -> (stored', putBucket b' mx outputs')
    in (stored'',outputs'')

run (State stored outputs [])     = (stored,outputs)
run (State stored outputs (i:is)) = case i of
    Input int bot -> let
        stored' = storeInput bot stored int
        in run (State stored' outputs is)
    Output bot sink1 sink2
        | not $ full bot stored -> run (State stored outputs (is++[i]))
        | otherwise             -> let
            value              = stored M.! bot
            (stored',outputs') = output value sink1 sink2 stored outputs
            in run (State stored' outputs' is)
    
result = run (State M.empty M.empty parsed)

matcher (_,(Holding (Just 17) (Just 61))) = True
matcher _                                 = False

fst $ head $ filter matcher $ M.toList (fst result)

Bot 93

In [4]:
outputs = snd result
head (outputs M.! 0) * head (outputs M.! 1) * head (outputs M.! 2)

47101